In [5]:
!pip install scikit-learn

In [6]:
import pandas as pd
import requests
import numpy as np
from sklearn.metrics import root_mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import root_mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
player_id = 355 
url_player = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
api_result_player = requests.get(url_player).json()
stats = api_result_player['history']
df_stats = pd.json_normalize(stats)
df_stats.index = np.arange(1, len(df_stats) + 1)
features = ['opponent_team', 'was_home', 'minutes', 'total_points', 'goals_scored', 'assists', 'expected_goals', 'expected_assists']
df_stats.columns.values

# features = ['element', 'opponent_team', 'was_home', 'total_points']

array(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals',
       'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'value', 'transfers_balance',
       'selected', 'transfers_in', 'transfers_out'], dtype=object)

In [17]:
df_stats = df_stats[features]
df_stats['avg_points'] = df_stats.total_points.rolling(window=3, closed = 'left', min_periods = 1).mean()

In [18]:
df_stats

,opponent_team,was_home,minutes,total_points,goals_scored,assists,expected_goals,expected_assists,avg_points
1,6,False,79,13,2,0,0.60,0.05,NaN
2,15,True,90,2,0,0,0.43,0.02,13.000000
3,17,False,90,4,1,0,1.93,0.01,7.500000
4,10,True,90,20,3,1,1.58,0.10,6.333333
5,19,False,90,6,1,0,2.56,0.03,8.666667
6,16,True,90,6,1,0,0.69,0.01,10.000000
7,20,False,90,2,0,0,0.06,0.03,10.666667
8,1,False,90,2,0,0,0.00,0.17,4.666667
9,5,True,90,8,1,0,0.26,0.00,3.333333
10,14,False,90,16,2,1,2.04,0.47,4.000000


In [19]:
df_stats.avg_points = df_stats.avg_points.fillna(1)

In [23]:
df_stats['avg_points'] = round(df_stats['avg_points'], 2)

In [24]:
df_stats

,opponent_team,was_home,minutes,total_points,goals_scored,assists,expected_goals,expected_assists,avg_points
1,6,False,79,13,2,0,0.60,0.05,1.00
2,15,True,90,2,0,0,0.43,0.02,13.00
3,17,False,90,4,1,0,1.93,0.01,7.50
4,10,True,90,20,3,1,1.58,0.10,6.33
5,19,False,90,6,1,0,2.56,0.03,8.67
6,16,True,90,6,1,0,0.69,0.01,10.00
7,20,False,90,2,0,0,0.06,0.03,10.67
8,1,False,90,2,0,0,0.00,0.17,4.67
9,5,True,90,8,1,0,0.26,0.00,3.33
10,14,False,90,16,2,1,2.04,0.47,4.00


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_stats[['avg_points']], df_stats['total_points'], test_size=0.3, random_state=0)

In [26]:
class MyBaselineEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.column_name]

    def predict(self, X):
        return self.transform(X)

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

pipeline_champion = Pipeline([
    ('classifier', MyBaselineEstimator('avg_points')),
])
pipeline_challenger = Pipeline([
    ('LinearRegression', LinearRegression())
     ])

In [30]:
def compare_two_pipelines_performance(pipeline_champion, pipeline_challenger,
                                      X_train, X_test, y_train, y_test):
    # Train pipeline1
    pipeline_champion.fit(X_train, y_train)
    y_pred_champion = pipeline_champion.predict(X_test)
    # Train pipeline2
    pipeline_challenger.fit(X_train, y_train)
    y_pred_challenger = pipeline_challenger.predict(X_test)
    # Compute the accuracy of pipeline1
    champion_accuracy = root_mean_squared_error(y_test, y_pred_champion)
    # Compute the accuracy of pipeline2
    challenger_accuracy = root_mean_squared_error(y_test, y_pred_challenger)
    print(f"pipeline_champion accuracy: {champion_accuracy}")
    print(f"pipeline_challenger accuracy: {challenger_accuracy}")
    return champion_accuracy, challenger_accuracy

In [31]:
compare_two_pipelines_performance(pipeline_champion, pipeline_challenger, X_train, X_test, y_train, y_test)

pipeline_champion accuracy: 5.964144951290168
pipeline_challenger accuracy: 5.414548839703626


(5.964144951290168, 5.414548839703626)

In [98]:
###  TO DO: DODAĆ DO df_players_stats KOLUMNĘ avg_points A NASTĘPNIE WYKONAĆ PORÓWNANIE PIPELINE

In [81]:
# Let's create dataframe df_players_stats including all the data on players performance in each game
d = {'element': [], 'opponent_team': [], 'was_home': [], 'minutes': [],'total_points': [], 'goals_scored': [], 'assists': [], 'expected_goals': [], 'expected_assists': []}
df_players_stats = pd.DataFrame(d)
columns = ['element', 'opponent_team', 'was_home', 'minutes', 'total_points', 'goals_scored', 'assists', 'expected_goals', 'expected_assists']

for i in range(1, 839):
    url = f'https://fantasy.premierleague.com/api/element-summary/{i}/'
    try:
        api_result = requests.get(url).json()
        stats = api_result['history']
        df_stats = pd.json_normalize(stats)
        df_stats.index = np.arange(1, len(df_stats) + 1)
        df_stats = df_stats.reindex(columns=columns)
        df_stats['avg_minutes'] = df_stats.minutes.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_stats['avg_points'] = df_stats.total_points.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_stats['avg_goals'] = df_stats.goals_scored.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_stats['avg_xG'] = df_stats.expected_goals.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_stats['avg_assists'] = df_stats.assists.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_stats['avg_xA'] = df_stats.expected_assists.rolling(window=3, closed = 'left', min_periods = 1).mean()
        df_players_stats = pd.concat([df_players_stats, df_stats])
    except Exception as e:
        print(f'Error fetching data for player {i}: {e}')
        continue

df_players_stats.index = np.arange(1, len(df_players_stats) + 1)

Error fetching data for player 612: 'history'


In [82]:
# After creatuib of df_players_stats dataframe let's clean the data and prepare it for further predictions
df_players_stats.avg_points = df_players_stats.avg_points.fillna(1)
df_players_stats.avg_minutes = df_players_stats.avg_minutes.fillna(0)
df_players_stats.avg_goals = df_players_stats.avg_goals.fillna(0)
df_players_stats.avg_xG = df_players_stats.avg_xG.fillna(0)
df_players_stats.avg_assists = df_players_stats.avg_assists.fillna(0)
df_players_stats.avg_xA = df_players_stats.avg_xA.fillna(0)
df_players_stats['avg_points'] = round(df_players_stats['avg_points'], 2)
df_players_stats['avg_minutes'] = round(df_players_stats['avg_minutes'], 2)
df_players_stats['avg_goals'] = round(df_players_stats['avg_goals'], 2)
df_players_stats['avg_xG'] = round(df_players_stats['avg_xG'], 2)
df_players_stats['avg_assists'] = round(df_players_stats['avg_assists'], 2)
df_players_stats['avg_xA'] = round(df_players_stats['avg_xA'], 2)
df_players_stats['was_home'] = df_players_stats['was_home'].astype('bool')
df_players_stats['opponent_team'] = df_players_stats['opponent_team'].astype('int')
df_players_stats['opponent_team'] = df_players_stats['opponent_team'].astype('string')
df_players_stats['element'] = df_players_stats['element'].astype('int64')

df_players_stats.describe()

,element,minutes,total_points,goals_scored,assists,avg_minutes,avg_points,avg_goals,avg_xG,avg_assists,avg_xA
count,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000,28709.000000
mean,390.530670,25.249399,1.054129,0.040127,0.035912,24.744730,1.060898,0.039101,0.039513,0.035027,0.024856
std,227.469389,37.515165,2.289895,0.219866,0.204278,34.369577,1.743372,0.145704,0.111755,0.130506,0.062664
min,1.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000
25%,194.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,388.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,582.000000,66.000000,1.000000,0.000000,0.000000,54.330000,1.330000,0.000000,0.020000,0.000000,0.020000
max,838.000000,90.000000,26.000000,4.000000,4.000000,90.000000,17.000000,2.330000,2.020000,2.000000,0.940000


In [83]:
#Column 'elements' has to many values to be one hot encoded. Let's join the 'element_type' column from df_players table to group players by their position.
df_players = pd.read_csv('data dumps/df_players.csv')
df_players = df_players.rename(columns = {"id" : "element"})
df_players.set_index(['element'], inplace = True)
df_players_stats = df_players_stats.join(df_players['element_type'], on = 'element', how = 'left')
df_players_stats['element_type'] = df_players_stats['element_type'].astype('string')
df_players_stats


In [133]:
# Save dataframe to csv for future use
df_players_stats.to_csv(path_or_buf='df_players_stats.csv', index = False)

In [161]:
#Let's implement one hot encoding on the 'element_type' column
df_players_stats = pd.read_csv('df_players_stats.csv')
oh_element_type = pd.get_dummies(df_players_stats['element_type'])
df_players_stats.drop('element_type', axis = 1, inplace = True)
df_players_stats = df_players_stats.join(oh_element_type)
df_players_stats = df_players_stats.rename(columns = {1 : 'element_type_1', 2 : 'element_type_2', 3 : 'element_type_3', 4 : 'element_type_4'})
df_players_stats['element_type_1'] = df_players_stats['element_type_1'].astype('int64')
df_players_stats['element_type_2'] = df_players_stats['element_type_2'].astype('int64')
df_players_stats['element_type_3'] = df_players_stats['element_type_3'].astype('int64')
df_players_stats['element_type_4'] = df_players_stats['element_type_4'].astype('int64')


In [171]:
#Let's do the same for the 'opponent_team' column
oh_opponent_team = pd.get_dummies(df_players_stats['opponent_team'])
df_players_stats.drop('opponent_team', axis = 1, inplace = True)
df_players_stats = df_players_stats.join(oh_opponent_team)
for i in range (1,21):
    df_players_stats = df_players_stats.rename(columns = {i : 'opponent_' + f'{i}'})
for i in range (1,21):
    df_players_stats[f'opponent_{i}'] = df_players_stats[f'opponent_{i}'].astype('int64')

In [172]:
df_players_stats

,element,was_home,minutes,total_points,goals_scored,assists,expected_goals,expected_assists,avg_minutes,avg_points,...,opponent_11,opponent_12,opponent_13,opponent_14,opponent_15,opponent_16,opponent_17,opponent_18,opponent_19,opponent_20
0,1,True,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,...,0,0,0,0,0,1,0,0,0,0
1,1,False,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
2,1,True,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
3,1,True,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,1,0,0,0,0,0,0
4,1,False,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28704,838,False,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
28705,838,True,5.0,1.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0,0,0,0,0,0,0,1,0,0
28706,838,True,0.0,0.0,0.0,0.0,0.0,0.0,1.67,0.33,...,0,0,0,0,0,0,0,0,1,0
28707,838,False,0.0,0.0,0.0,0.0,0.0,0.0,1.67,0.33,...,0,0,0,0,0,1,0,0,0,0


In [174]:
df_players_stats['was_home'] = df_players_stats['was_home'].astype('int64')

In [175]:
# Save dataframe to csv for future use
df_players_stats.to_csv(path_or_buf='df_players_stats_oh.csv', index = False)

In [ ]:
#TO DO: ZASTOSOWAĆ VECTOR ASSEMBLER I STANDARD SCALER ZGODNIE Z ZAJĘCIAMI